In [1]:
import os
os.chdir("..")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel(os.path.join("ratio_analysis_plots", "d_processed.xlsx"))

In [3]:
data

,dataset,ml_name,method,lambda,k,Avg.e^2
0,adult,KNN,mean_v1,14.569371,1.933125e-06,0.424207
1,adult,KNN,mean_v2,-3.832102,3.892365e+00,0.332409
2,adult,KNN,similar_v1,6.629730,1.111963e-03,0.147048
3,adult,KNN,similar_v2,5.332352,2.483361e-03,0.241792
4,adult,KNN,multi_v1,5.095211,3.511095e-03,0.090114
...,...,...,...,...,...,...
211,bank,MLP,mean_v2,-8.510056,7.599501e+01,0.556829
212,bank,MLP,similar_v1,-0.694158,6.901930e-02,0.452284
213,bank,MLP,similar_v2,-7.076779,1.754112e+01,0.527676
214,bank,MLP,multi_v1,18.064996,4.123700e-09,0.623496


### Analysis of Output  
Dataset is the major factor of exponential curve fit and curve steepness

#### Groupby ML

In [4]:
data.groupby("ml_name").agg({'Avg.e^2':'median'}).sort_values(by="Avg.e^2")

,Avg.e^2
ml_name,
LinearSVC,0.325652
LogReg,0.325905
KNN,0.387774
MLP,0.403259
Tree,0.417961
Forest,0.423837


In [5]:
data.groupby("ml_name").agg({'lambda':'median'}).sort_values(by="lambda")

,lambda
ml_name,
KNN,0.353970
LogReg,0.384772
LinearSVC,0.600193
MLP,1.727437
Tree,1.897911
Forest,3.016339


#### Groupby Dataset

In [6]:
data.groupby("dataset").agg({'Avg.e^2':'median'}).sort_values(by="Avg.e^2")

,Avg.e^2
dataset,
compas,0.148974
adult,0.247918
communities,0.340193
titanic,0.448134
german,0.481169
bank,0.515395


In [7]:
data.groupby("dataset").agg({'lambda':'median'}).sort_values(by="lambda")

,lambda
dataset,
titanic,-1.184967
bank,-0.785825
german,-0.611679
communities,2.770809
adult,5.719160
compas,6.417708


#### Groupby Imputation Method

In [8]:
data.groupby("method").agg({'Avg.e^2':'median'}).sort_values(by="Avg.e^2")

,Avg.e^2
method,
similar_v1,0.273038
multi_v1,0.280543
multi_v2,0.370827
mean_v1,0.406623
similar_v2,0.421221
mean_v2,0.470482


In [9]:
data.groupby("method").agg({'lambda':'median'}).sort_values(by="lambda")

,lambda
method,
mean_v2,-0.593358
similar_v2,0.539437
similar_v1,2.004557
multi_v1,2.121350
mean_v1,2.244799
multi_v2,2.481914


### Analysis of Datasets  
1. Correlation to the protected attribute can affect the initial bias, but not related to curve steepness

In [10]:
from utils.data import create_adult_dataset, create_bank_dataset
from utils.data import create_communities_dataset, create_compas_dataset
from utils.data import create_german_dataset, create_titanic_dataset
from sklearn.preprocessing import LabelEncoder

In [11]:
def drop_na(data):
    data = data.copy()
    tmp_concat = pd.concat([data.X, pd.DataFrame(data.y, columns=["_TARGET_"])], axis=1)
    tmp_concat.dropna(inplace=True)
    tmp_concat.reset_index(drop=True, inplace=True)
    data.X = tmp_concat.drop(columns=["_TARGET_"]).copy()
    data.y = tmp_concat["_TARGET_"].copy().to_numpy().ravel()
    return data

In [12]:
def convert_protected(data):
    data = data.copy()
    encoder = LabelEncoder()
    for feature in data.protected_features:
        data.X[feature] = encoder.fit_transform(data.X[feature])
    return data, encoder

In [13]:
def concat(data):
    data = data.copy()
    return pd.concat([data.X, pd.DataFrame(data.y, columns=["_TARGET_"])], axis=1)

### Adult Dataset

In [14]:
data_adult = create_adult_dataset()
data_adult = drop_na(data_adult)
data_adult, encoder = convert_protected(data_adult)
data_a = concat(data_adult)
data_adult.protected_features

['sex']

In [15]:
encoder.classes_

array([' Female', ' Male'], dtype=object)

In [16]:
data_a.describe()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,_TARGET_
count,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000,30718.000000
mean,38.443584,3.100332,10.328602,10.130314,2.583143,5.967088,1.417247,3.670161,0.676737,40.949313,0.249040
std,13.118227,1.139557,3.809556,2.562469,1.495674,4.025999,1.601399,0.844063,0.467730,11.985382,0.432464
min,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,3.000000,9.000000,9.000000,2.000000,2.000000,0.000000,4.000000,0.000000,40.000000,0.000000
50%,37.000000,3.000000,11.000000,10.000000,2.000000,6.000000,1.000000,4.000000,1.000000,40.000000,0.000000
75%,47.000000,3.000000,12.000000,13.000000,4.000000,9.000000,3.000000,4.000000,1.000000,45.000000,0.000000
max,90.000000,7.000000,15.000000,16.000000,6.000000,13.000000,5.000000,4.000000,1.000000,99.000000,1.000000


In [17]:
data_a["_TARGET_"].value_counts()

0    23068
1     7650
Name: _TARGET_, dtype: int64

In [18]:
data_a[data_adult.protected_features[0]].value_counts()

1    20788
0     9930
Name: sex, dtype: int64

In [19]:
data_a.groupby(["_TARGET_", data_adult.protected_features[0]]).count()

age  workclass  education  education-num  marital-status  \
_TARGET_ sex                                                               
0        0     8803       8803       8803           8803            8803   
         1    14265      14265      14265          14265           14265   
1        0     1127       1127       1127           1127            1127   
         1     6523       6523       6523           6523            6523   

              occupation  relationship   race  hours-per-week  
_TARGET_ sex                                                   
0        0          8803          8803   8803            8803  
         1         14265         14265  14265           14265  
1        0          1127          1127   1127            1127  
         1          6523          6523   6523            6523

In [20]:
data_a.groupby(["_TARGET_", data_adult.protected_features[0]]).nunique()

age  workclass  education  education-num  marital-status  \
_TARGET_ sex                                                             
0        0     71          7         16             16               7   
         1     71          7         16             16               7   
1        0     53          6         14             14               7   
         1     65          6         15             15               7   

              occupation  relationship  race  sex  hours-per-week  _TARGET_  
_TARGET_ sex                                                                 
0        0            13             6     5    1              78         1  
         1            14             6     5    1              90         1  
1        0            13             5     5    1              53         1  
         1            13             6     5    1              76         1

In [21]:
data_a.corr()[data_adult.protected_features[0]]

age               0.082117
workclass         0.072763
education        -0.028155
education-num     0.007443
marital-status   -0.120360
occupation        0.062068
relationship     -0.585792
race              0.086147
sex               1.000000
hours-per-week    0.230321
_TARGET_          0.216626
Name: sex, dtype: float64

### Compas Dataset

In [22]:
data_compas = create_compas_dataset()
data_compas = drop_na(data_compas)
data_compas, encoder = convert_protected(data_compas)
data_c = concat(data_compas)
data_compas.protected_features

['race']

In [23]:
encoder.classes_

array(['African-American', 'Asian', 'Caucasian', 'Hispanic',
       'Native American', 'Other'], dtype=object)

In [24]:
data_c.describe()

,age,age_cat,c_charge_degree,priors_count,juv_misd_count,juv_fel_count,juv_other_count,days_b_screening_arrest,sex,race,length_of_stay,_TARGET_
count,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000,9388.000000
mean,34.760865,0.642522,0.350980,3.022262,0.078718,0.058266,0.098743,-1.800597,0.794099,1.274819,523.774782,0.339476
std,11.813824,0.811018,0.477302,4.585775,0.463982,0.451685,0.467258,5.024736,0.404380,1.454650,1725.088680,0.473557
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-30.000000,0.000000,0.000000,-11.704722,0.000000
25%,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,21.388889,0.000000
50%,32.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,1.000000,29.148889,0.000000
75%,42.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,2.000000,168.065069,1.000000
max,96.000000,2.000000,1.000000,38.000000,13.000000,20.000000,11.000000,30.000000,1.000000,5.000000,51670.880278,1.000000


In [25]:
data_c["_TARGET_"].value_counts()

0    6201
1    3187
Name: _TARGET_, dtype: int64

In [26]:
data_c[data_compas.protected_features[0]].value_counts()

0    4672
2    3253
3     817
5     571
1      48
4      27
Name: race, dtype: int64

In [27]:
data_c.groupby(["_TARGET_", data_compas.protected_features[0]]).count()

age  age_cat  c_charge_degree  priors_count  juv_misd_count  \
_TARGET_ race                                                                 
0        0     2781     2781             2781          2781            2781   
         1       38       38               38            38              38   
         2     2323     2323             2323          2323            2323   
         3      609      609              609           609             609   
         4       19       19               19            19              19   
         5      431      431              431           431             431   
1        0     1891     1891             1891          1891            1891   
         1       10       10               10            10              10   
         2      930      930              930           930             930   
         3      208      208              208           208             208   
         4        8        8                8             8               8   
         5      140      140              140           140             140   

               juv_fel_count  juv_other_count  days_b_screening_arrest   sex  \
_TARGET_ race                                                                  
0        0              2781             2781                     2781  2781   
         1                38               38                       38    38   
         2              2323             2323                     2323  2323   
         3               609              609                      609   609   
         4                19               19                       19    19   
         5               431              431                      431   431   
1        0              1891             1891                     1891  1891   
         1                10               10                       10    10   
         2               930              930                      930   930   
         3               208              208                      208   208   
         4                 8                8                        8     8   
         5               140              140                      140   140   

               length_of_stay  
_TARGET_ race                  
0        0               2781  
         1                 38  
         2               2323  
         3                609  
         4                 19  
         5                431  
1        0               1891  
         1                 10  
         2                930  
         3                208  
         4                  8  
         5                140

In [28]:
data_c.groupby(["_TARGET_", data_compas.protected_features[0]]).nunique()

age  age_cat  c_charge_degree  priors_count  juv_misd_count  \
_TARGET_ race                                                                
0        0      59        3                2            30               8   
         1      24        3                2             3               1   
         2      62        3                2            25               7   
         3      54        3                2            18               4   
         4      16        3                2             7               1   
         5      51        3                2            13               2   
1        0      52        3                2            35              10   
         1      10        3                2             7               2   
         2      52        3                2            29               4   
         3      41        3                2            18               4   
         4       7        3                2             7               2   
         5      41        3                2            15               4   

               juv_fel_count  juv_other_count  days_b_screening_arrest  sex  \
_TARGET_ race                                                                 
0        0                 8                8                       43    2   
         1                 1                1                        6    2   
         2                 5                7                       42    2   
         3                 3                6                       28    2   
         4                 2                2                        3    2   
         5                 4                3                       17    2   
1        0                 9                7                       48    2   
         1                 1                2                        2    2   
         2                 7                7                       45    2   
         3                 3                3                       24    2   
         4                 2                2                        4    2   
         5                 2                3                        7    2   

               race  length_of_stay  _TARGET_  
_TARGET_ race                                  
0        0        1            2765         1  
         1        1              38         1  
         2        1            2313         1  
         3        1             607         1  
         4        1              19         1  
         5        1             431         1  
1        0        1            1885         1  
         1        1              10         1  
         2        1             928         1  
         3        1             208         1  
         4        1               8         1  
         5        1             140         1

In [29]:
data_c.corr()[data_compas.protected_features[0]]

age                        0.137085
age_cat                   -0.023542
c_charge_degree            0.099344
priors_count              -0.206018
juv_misd_count            -0.074672
juv_fel_count             -0.048369
juv_other_count           -0.050273
days_b_screening_arrest   -0.041953
sex                       -0.016440
race                       1.000000
length_of_stay            -0.058908
_TARGET_                  -0.129416
Name: race, dtype: float64

### Communities Dataset

In [30]:
data_communities = create_communities_dataset()
data_communities = drop_na(data_communities)
data_communities, encoder = convert_protected(data_communities)
data_co = concat(data_communities)
data_communities.protected_features

['race_c']

In [31]:
encoder.classes_

array(['asian', 'black', 'hispanic', 'white'], dtype=object)

In [32]:
data_co.describe()

,population,householdsize,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,...,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,race_c,_TARGET_
count,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,...,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000,1993.000000
mean,0.057612,0.463437,0.424210,0.493914,0.336297,0.423086,0.064104,0.696618,0.361259,0.558314,...,0.608776,0.534967,0.626322,0.651470,0.065243,0.232910,0.161741,0.094099,2.591069,0.355243
std,0.126935,0.163747,0.155234,0.143584,0.166540,0.179196,0.128280,0.444648,0.209327,0.182820,...,0.204314,0.181360,0.200520,0.198253,0.109485,0.203127,0.229099,0.240379,0.851918,0.478707
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.010000,0.350000,0.340000,0.410000,0.250000,0.300000,0.000000,0.000000,0.200000,0.440000,...,0.470000,0.420000,0.520000,0.560000,0.020000,0.100000,0.020000,0.000000,3.000000,0.000000
50%,0.020000,0.440000,0.400000,0.480000,0.290000,0.420000,0.030000,1.000000,0.320000,0.560000,...,0.630000,0.540000,0.670000,0.700000,0.040000,0.170000,0.070000,0.000000,3.000000,0.000000
75%,0.050000,0.540000,0.470000,0.540000,0.360000,0.530000,0.070000,1.000000,0.490000,0.690000,...,0.770000,0.660000,0.770000,0.790000,0.070000,0.280000,0.190000,0.000000,3.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,1.000000


In [33]:
data_co["_TARGET_"].value_counts()

0    1285
1     708
Name: _TARGET_, dtype: int64

In [34]:
data_co[data_communities.protected_features[0]].value_counts()

3    1572
1     218
2     115
0      88
Name: race_c, dtype: int64

In [35]:
data_co.groupby(["_TARGET_", data_communities.protected_features[0]]).count()

population  householdsize  agePct12t21  agePct12t29  \
_TARGET_ race_c                                                        
0        0               43             43           43           43   
         1               30             30           30           30   
         2               24             24           24           24   
         3             1188           1188         1188         1188   
1        0               45             45           45           45   
         1              188            188          188          188   
         2               91             91           91           91   
         3              384            384          384          384   

                 agePct16t24  agePct65up  numbUrban  pctUrban  medIncome  \
_TARGET_ race_c                                                            
0        0                43          43         43        43         43   
         1                30          30         30        30         30   
         2                24          24         24        24         24   
         3              1188        1188       1188      1188       1188   
1        0                45          45         45        45         45   
         1               188         188        188       188        188   
         2                91          91         91        91         91   
         3               384         384        384       384        384   

                 pctWWage  ...  NumStreet  PctForeignBorn  PctBornSameState  \
_TARGET_ race_c            ...                                                
0        0             43  ...         43              43                43   
         1             30  ...         30              30                30   
         2             24  ...         24              24                24   
         3           1188  ...       1188            1188              1188   
1        0             45  ...         45              45                45   
         1            188  ...        188             188               188   
         2             91  ...         91              91                91   
         3            384  ...        384             384               384   

                 PctSameHouse85  PctSameCity85  PctSameState85  LandArea  \
_TARGET_ race_c                                                            
0        0                   43             43              43        43   
         1                   30             30              30        30   
         2                   24             24              24        24   
         3                 1188           1188            1188      1188   
1        0                   45             45              45        45   
         1                  188            188             188       188   
         2                   91             91              91        91   
         3                  384            384             384       384   

                 PopDens  PctUsePubTrans  LemasPctOfficDrugUn  
_TARGET_ race_c                                                
0        0            43              43                   43  
         1            30              30                   30  
         2            24              24                   24  
         3          1188            1188                 1188  
1        0            45              45                   45  
         1           188             188                  188  
         2            91              91                   91  
         3           384             384                  384  

[8 rows x 96 columns]

In [36]:
data_co.groupby(["_TARGET_", data_communities.protected_features[0]]).nunique()

population  householdsize  agePct12t21  agePct12t29  \
_TARGET_ race_c                                                        
0        0               17             32           25           28   
         1                9             22           22           20   
         2                7             20           19           16   
         3               25             81           85           83   
1        0               25             35           25           27   
         1               47             48           50           46   
         2               24             38           35           30   
         3               44             62           63           59   

                 agePct16t24  agePct65up  numbUrban  pctUrban  medIncome  \
_TARGET_ race_c                                                            
0        0                24          32         20         2         28   
         1                23          25          8         3         23   
         2                17          16          7         2         16   
         3                87          93         28        64         96   
1        0                24          26         26         2         31   
         1                52          53         48         6         39   
         2                29          34         25         3         36   
         3                59          78         43        10         58   

                 pctWWage  ...  PctBornSameState  PctSameHouse85  \
_TARGET_ race_c            ...                                     
0        0             33  ...                23              32   
         1             25  ...                24              19   
         2             19  ...                22              23   
         3             90  ...                99              99   
1        0             29  ...                25              34   
         1             57  ...                63              53   
         2             43  ...                45              45   
         3             82  ...                87              74   

                 PctSameCity85  PctSameState85  LandArea  PopDens  \
_TARGET_ race_c                                                     
0        0                  35              32        12       34   
         1                  25              22        11       21   
         2                  16              19         6       16   
         3                  96              92        36       76   
1        0                  30              29        19       33   
         1                  60              63        40       54   
         2                  41              43        17       49   
         3                  85              80        43       67   

                 PctUsePubTrans  LemasPctOfficDrugUn  race_c  _TARGET_  
_TARGET_ race_c                                                         
0        0                   26                    8       1         1  
         1                   16                    3       1         1  
         2                   12                    3       1         1  
         3                   90                   53       1         1  
1        0                   30                   15       1         1  
         1                   58                   43       1         1  
         2                   41                   16       1         1  
         3                   56                   49       1         1  

[8 rows x 98 columns]

In [37]:
data_co.corr()[data_communities.protected_features[0]].sort_values().head(20)

PctIlleg              -0.550950
PctPersDenseHous      -0.478658
PctLargHouseFam       -0.431739
PctRecImmig10         -0.430873
PctRecImmig8          -0.420352
pctWPubAsst           -0.404295
PctRecImmig5          -0.393945
_TARGET_              -0.384646
PctLargHouseOccup     -0.381241
PctForeignBorn        -0.380517
PctRecentImmig        -0.380115
PctNotSpeakEnglWell   -0.356021
PctPopUnderPov        -0.335970
NumIlleg              -0.329182
PersPerRentOccHous    -0.327487
NumImmig              -0.322164
PctUnemployed         -0.309424
PopDens               -0.300363
PersPerFam            -0.298405
NumUnderPov           -0.294006
Name: race_c, dtype: float64

In [38]:
data_co.corr()[data_communities.protected_features[0]].sort_values().tail(20)

agePct65up          0.093867
pctWFarmSelf        0.095757
PctOccupMgmtProf    0.096813
PctBornSameState    0.102681
PctWorkMom          0.110446
medIncome           0.128469
perCapInc           0.132593
PctEmploy           0.145985
medFamInc           0.154943
MedNumBR            0.163703
pctWRetire          0.175166
PctHousOwnOcc       0.270120
PctPersOwnOccup     0.316089
pctWInvInc          0.344894
PctSpeakEnglOnly    0.358004
PctYoungKids2Par    0.379854
PctTeen2Par         0.411145
PctFam2Par          0.419595
PctKids2Par         0.465343
race_c              1.000000
Name: race_c, dtype: float64

### Titanic Dataset

In [39]:
data_titanic = create_titanic_dataset()
data_titanic = drop_na(data_titanic)
data_titanic, encoder = convert_protected(data_titanic)
data_t = concat(data_titanic)
data_titanic.protected_features

['Sex']

In [40]:
encoder.classes_

array(['female', 'male'], dtype=object)

In [41]:
data_t.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,_TARGET_
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,2.240169,0.636236,29.642093,0.514045,0.432584,34.567251,1.595506,0.404494
std,0.836854,0.481420,14.492933,0.930692,0.854181,52.938648,0.779038,0.491139
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,20.000000,0.000000,0.000000,8.050000,2.000000,0.000000
50%,2.000000,1.000000,28.000000,0.000000,0.000000,15.645850,2.000000,0.000000
75%,3.000000,1.000000,38.000000,1.000000,1.000000,33.000000,2.000000,1.000000
max,3.000000,1.000000,80.000000,5.000000,6.000000,512.329200,2.000000,1.000000


In [42]:
data_t["_TARGET_"].value_counts()

0    424
1    288
Name: _TARGET_, dtype: int64

In [43]:
data_t[data_titanic.protected_features[0]].value_counts()

1    453
0    259
Name: Sex, dtype: int64

In [44]:
data_t.groupby(["_TARGET_", data_titanic.protected_features[0]]).count()

Pclass  Age  SibSp  Parch  Fare  Embarked
_TARGET_ Sex                                           
0        0        64   64     64     64    64        64
         1       360  360    360    360   360       360
1        0       195  195    195    195   195       195
         1        93   93     93     93    93        93

In [45]:
data_t.groupby(["_TARGET_", data_titanic.protected_features[0]]).nunique()

Pclass  Sex  Age  SibSp  Parch  Fare  Embarked  _TARGET_
_TARGET_ Sex                                                          
0        0         3    1   38      6      7    38         3         1
         1         3    1   74      6      6   140         3         1
1        0         3    1   54      5      5   114         3         1
         1         3    1   49      4      3    59         3         1

In [46]:
data_t.corr()[data_titanic.protected_features[0]]

Pclass      0.150826
Sex         1.000000
Age         0.099037
SibSp      -0.106296
Parch      -0.249543
Fare       -0.182457
Embarked    0.109639
_TARGET_   -0.536762
Name: Sex, dtype: float64

### German Credit Dataset

In [47]:
data_german = create_german_dataset()
data_german = drop_na(data_german)
data_german, encoder = convert_protected(data_german)
data_g = concat(data_german)
data_german.protected_features

['Age']

In [48]:
encoder.classes_

array(['elder', 'young'], dtype=object)

In [49]:
data_g.describe()

,Status_account,Duration_month,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,Personal_status,Debtors_guarantors,...,Property,Age,Installment_plans,Housing,Number_credits,Job,Num_liable_people,Telephone,Foreign,_TARGET_
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,1.577000,20.903000,2.54500,3.277000,3271.258000,1.105000,2.384000,2.973000,1.68200,0.145000,...,1.358000,0.190000,1.675000,0.929000,1.407000,1.904000,1.155000,0.404000,0.037000,0.300000
std,1.257638,12.058814,1.08312,2.739302,2822.736876,1.580023,1.208306,1.118715,0.70808,0.477706,...,1.050209,0.392497,0.705601,0.531264,0.577654,0.653614,0.362086,0.490943,0.188856,0.458487
min,0.000000,4.000000,0.00000,0.000000,250.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,12.000000,2.00000,1.000000,1365.500000,0.000000,2.000000,2.000000,1.00000,0.000000,...,0.000000,0.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
50%,1.000000,18.000000,2.00000,3.000000,2319.500000,0.000000,2.000000,3.000000,2.00000,0.000000,...,1.000000,0.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,24.000000,4.00000,4.000000,3972.250000,2.000000,4.000000,4.000000,2.00000,0.000000,...,2.000000,0.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000
max,3.000000,72.000000,4.00000,9.000000,18424.000000,4.000000,4.000000,4.000000,3.00000,2.000000,...,3.000000,1.000000,2.000000,2.000000,4.000000,3.000000,2.000000,1.000000,1.000000,1.000000


In [50]:
data_g["_TARGET_"].value_counts()

0    700
1    300
Name: _TARGET_, dtype: int64

In [51]:
data_g[data_german.protected_features[0]].value_counts()

0    810
1    190
Name: Age, dtype: int64

In [52]:
data_g.groupby(["_TARGET_", data_german.protected_features[0]]).count()

Status_account  Duration_month  Credit_history  Purpose  \
_TARGET_ Age                                                            
0        0               590             590             590      590   
         1               110             110             110      110   
1        0               220             220             220      220   
         1                80              80              80       80   

              Credit_amount  Savings_account  Employment_since  \
_TARGET_ Age                                                     
0        0              590              590               590   
         1              110              110               110   
1        0              220              220               220   
         1               80               80                80   

              Installment_rate  Personal_status  Debtors_guarantors  \
_TARGET_ Age                                                          
0        0                 590              590                 590   
         1                 110              110                 110   
1        0                 220              220                 220   
         1                  80               80                  80   

              Residence_since  Property  Installment_plans  Housing  \
_TARGET_ Age                                                          
0        0                590       590                590      590   
         1                110       110                110      110   
1        0                220       220                220      220   
         1                 80        80                 80       80   

              Number_credits  Job  Num_liable_people  Telephone  Foreign  
_TARGET_ Age                                                              
0        0               590  590                590        590      590  
         1               110  110                110        110      110  
1        0               220  220                220        220      220  
         1                80   80                 80         80       80

In [53]:
data_g.groupby(["_TARGET_", data_german.protected_features[0]]).nunique()

Status_account  Duration_month  Credit_history  Purpose  \
_TARGET_ Age                                                            
0        0                 4              31               5       10   
         1                 4              19               5        8   
1        0                 4              21               5        9   
         1                 4              18               5        9   

              Credit_amount  Savings_account  Employment_since  \
_TARGET_ Age                                                     
0        0              555                5                 5   
         1              109                5                 5   
1        0              219                5                 5   
         1               78                5                 5   

              Installment_rate  Personal_status  Debtors_guarantors  ...  \
_TARGET_ Age                                                         ...   
0        0                   4                4                   3  ...   
         1                   4                4                   3  ...   
1        0                   4                4                   3  ...   
         1                   4                3                   3  ...   

              Property  Age  Installment_plans  Housing  Number_credits  Job  \
_TARGET_ Age                                                                   
0        0           4    1                  3        3               4    4   
         1           4    1                  3        3               3    4   
1        0           4    1                  3        3               4    4   
         1           4    1                  3        3               2    4   

              Num_liable_people  Telephone  Foreign  _TARGET_  
_TARGET_ Age                                                   
0        0                    2          2        2         1  
         1                    2          2        2         1  
1        0                    2          2        2         1  
         1                    2          2        1         1  

[4 rows x 21 columns]

In [54]:
data_g.corr()[data_german.protected_features[0]]

Status_account       -0.137146
Duration_month       -0.007946
Credit_history       -0.114317
Purpose               0.005931
Credit_amount        -0.045989
Savings_account      -0.058027
Employment_since     -0.175101
Installment_rate     -0.061256
Personal_status      -0.077726
Debtors_guarantors    0.018419
Residence_since      -0.012824
Property             -0.080186
Age                   1.000000
Installment_plans     0.046084
Housing              -0.290479
Number_credits       -0.138322
Job                  -0.112219
Num_liable_people    -0.165169
Telephone            -0.164986
Foreign              -0.054422
_TARGET_              0.127938
Name: Age, dtype: float64

### Bank Dataset

In [55]:
data_bank = create_bank_dataset()
data_bank = drop_na(data_bank)
data_bank, encoder = convert_protected(data_bank)
data_b = concat(data_bank)
data_bank.protected_features

['age']

In [56]:
encoder.classes_

array(['elder', 'young'], dtype=object)

In [57]:
data_b.describe()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,_TARGET_
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,0.332065,4.339762,1.167725,1.224813,0.018027,1362.272058,0.555838,0.160226,0.640242,15.806419,5.523014,258.163080,2.763841,40.197828,0.580323,2.559974,0.116985
std,0.470959,3.272657,0.608230,0.747997,0.133049,3044.765829,0.496878,0.366820,0.897951,8.322476,3.006911,257.527812,3.098021,100.128746,2.303441,0.989059,0.321406
min,0.000000,0.000000,0.000000,0.000000,0.000000,-8019.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,1.000000,0.000000,72.000000,0.000000,0.000000,0.000000,8.000000,3.000000,103.000000,1.000000,-1.000000,0.000000,3.000000,0.000000
50%,0.000000,4.000000,1.000000,1.000000,0.000000,448.000000,1.000000,0.000000,0.000000,16.000000,6.000000,180.000000,2.000000,-1.000000,0.000000,3.000000,0.000000
75%,1.000000,7.000000,2.000000,2.000000,0.000000,1428.000000,1.000000,0.000000,2.000000,21.000000,8.000000,319.000000,3.000000,-1.000000,0.000000,3.000000,0.000000
max,1.000000,11.000000,2.000000,3.000000,1.000000,102127.000000,1.000000,1.000000,2.000000,31.000000,11.000000,4918.000000,63.000000,871.000000,275.000000,3.000000,1.000000


In [58]:
data_b["_TARGET_"].value_counts()

0    39922
1     5289
Name: _TARGET_, dtype: int64

In [59]:
data_b[data_bank.protected_features[0]].value_counts()

0    30198
1    15013
Name: age, dtype: int64

In [60]:
data_b.groupby(["_TARGET_", data_bank.protected_features[0]]).count()

job  marital  education  default  balance  housing   loan  \
_TARGET_ age                                                                
0        0    26889    26889      26889    26889    26889    26889  26889   
         1    13033    13033      13033    13033    13033    13033  13033   
1        0     3309     3309       3309     3309     3309     3309   3309   
         1     1980     1980       1980     1980     1980     1980   1980   

              contact    day  month  duration  campaign  pdays  previous  \
_TARGET_ age                                                               
0        0      26889  26889  26889     26889     26889  26889     26889   
         1      13033  13033  13033     13033     13033  13033     13033   
1        0       3309   3309   3309      3309      3309   3309      3309   
         1       1980   1980   1980      1980      1980   1980      1980   

              poutcome  
_TARGET_ age            
0        0       26889  
         1       13033  
1        0        3309  
         1        1980

In [61]:
data_b.groupby(["_TARGET_", data_bank.protected_features[0]]).nunique()

age  job  marital  education  default  balance  housing  loan  \
_TARGET_ age                                                                  
0        0      1   12        3          4        2     5980        2     2   
         1      1   12        3          4        2     3657        2     2   
1        0      1   12        3          4        2     1777        2     2   
         1      1   12        3          4        2     1159        2     2   

              contact  day  month  duration  campaign  pdays  previous  \
_TARGET_ age                                                             
0        0          3   31     12      1191        44    443        36   
         1          3   31     12       976        43    388        34   
1        0          3   31     12      1180        20    346        22   
         1          3   31     12       913        18    266        22   

              poutcome  _TARGET_  
_TARGET_ age                      
0        0           4         1  
         1           4         1  
1        0           4         1  
         1           4         1

In [62]:
data_b.corr()[data_bank.protected_features[0]]

age          1.000000
job          0.043744
marital      0.380677
education    0.089655
default      0.008954
balance     -0.053570
housing      0.074313
loan         0.000707
contact     -0.065795
day          0.012924
month        0.014213
duration     0.017457
campaign    -0.021444
pdays        0.020864
previous     0.004824
poutcome    -0.010869
_TARGET_     0.032689
Name: age, dtype: float64